# Missing Data Periods to JSON

In [1]:
import dataclient
import pandas as pd
import configparser
import json

import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use("seaborn-notebook")
%matplotlib inline
import pprint

import sys
sys.path.append("..")
from Wrapper import *


import ipywidgets as widgets
import plotly as py
import plotly.graph_objs as go

#to install plotly extension for jupyter lab see: https://github.com/jupyterlab/jupyter-renderers/tree/master/packages/plotly-extension

from scipy import special

In [2]:
mdal_obj = Import_MDAL()

start = '2016-01-01T00:00:00-08:00' #'2016-01-01T00:00:00Z' for UTC
end = '2019-02-01T00:00:00-08:00'# '2019-02-01T00:00:00Z' for UTC

site_list = ['avenal-public-works-yard',
    'avenal-movie-theatre',
    'avenal-recreation-center',
    'avenal-animal-shelter',
    'avenal-veterans-hall',
    'south-berkeley-senior-center',
    'north-berkeley-senior-center',
    'hayward-station-1',
    'hayward-station-8',
    'orinda-community-center',
    'ciee',
    'berkeley-corporate-yard']



In [3]:
missing_energy_data={}
missing_weather_data={}
missing_tstat_data={}

"""

Input Parameters:
obj: object returned from above MDAL Query
dictionary: pick dictionary that corresponds with data type
low_bound: all data <= this value will be interpreted as missing data
high_bound: all data > this value will be interpreted as missing data


Returns a JSON file with a dictionary with the missing data events (start and end)
        for each column in the inputted object

"""

def event_duration(obj,dictionary,low_bound=None, high_bound=None):
    clean_data_obj = Clean_Data(obj.df)
    dictionary=clean_data_obj.event_duration(obj, dictionary, low_bound, high_bound)
    return dictionary

### Missing Thermostat Data periods 

In [4]:
for site in site_list:
    try:
        obj_tstat = mdal_obj.get_tstat(site=site, start=start, end=end, var='tstat_state', agg='MEAN', window='15min', aligned=True, return_names=True)
        event_duration(obj_tstat, dictionary=missing_tstat_data)
    except:
        print('no data in',site)
        
#Comment out below to not generate json files        
# with open('missing_tstat_data.json',"w") as write_file:
#     json.dump(missing_tstat_data, write_file)

### Missing Energy Data periods 

In [5]:
for site in site_list:
    obj2 =  mdal_obj.get_meter(site=site, start=start, end=end, var='meter', agg='MEAN', window='15min', aligned=True, return_names=True)
    event_duration(obj2, dictionary=missing_energy_data)

#Comment out below to not generate json files  
with open('missing_energy_data.json',"w") as write_file:
    json.dump(missing_energy_data, write_file)

### Missing Weather Data periods 

In [6]:
for site in site_list:
    obj3 = mdal_obj.get_weather(site=site, start=start, end=end, var='weather', agg='MEAN', window='15min', aligned=True, return_names=True)
    event_duration(obj3, dictionary=missing_weather_data, low_bound=32, high_bound=110)

#Comment out below to not generate json files  
with open('missing_weather_data.json',"w") as write_file:
    json.dump(missing_weather_data, write_file)